# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2022-spring@googlegroups.com

# Download data
If the Google Drive links below do not work, you can download data from [Kaggle](https://www.kaggle.com/t/a3ebd5b5542f0f55e828d4f00de8e59a), and upload data manually to the workspace.

In [1]:
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS
To: /content/covid.train.csv
100% 2.49M/2.49M [00:00<00:00, 238MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg
To: /content/covid.test.csv
100% 993k/993k [00:00<00:00, 172MB/s]


# Import packages

In [2]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [3]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [4]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [5]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            #可以先接一个batch_norm层吗?
            # 不用，batchnorm只是对一个批次做normal，可以直接对所有的数据输入做归一化处理
            # nn.BatchNorm1d(input_dim),
            # 这里宽度是不是愈多愈好？
            # 这里深度是不是愈多愈好？
            # 由于数据有明显的线性关系，因此增加深度并没有什么用
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)  # (B, 1) -> (B)
        return x


# Feature Selection
Choose features you deem useful by modifying the function below.

In [7]:
'''Selects useful features to perform regression'''
def select_feat(train_data, valid_data, test_data, select_all=True):
    y_train, y_valid = train_data[:, -1], valid_data[:, -1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        # feat_idx = []
        # for i in range(1, 117):
        #     needJump = False
        #     for j in range(4):
        #         if i == (37+j*16+10): #shop
        #             needJump = True
        #         # if i == (37+j*16+9):  #restaurant
        #         #     needJump = True
        #         if i == (37+j*16+8):  #spent_time
        #             needJump = True
        #     if needJump is True:
        #         continue
        #     feat_idx.append(i)
        # print('fead_idx', feat_idx)
        #TODO: Select suitable feature columns.

        # features = pd.read_csv('./covid.train.csv')
        # x_data, y_data = features.iloc[:, 0:117], features.iloc[:, 117]
        # k = 24
        # selector = SelectKBest(score_func=f_regression, k=k)
        # result = selector.fit(x_data, y_data)
        # idx = np.argsort(result.scores_)[::-1]
        # feat_idx = list(np.sort(idx[:k]))
        train_data_df = pd.read_csv('./covid.train.csv')
        # 查看各个属性与最后预测值的线性相关性
        corr_df = train_data_df.corr()
        corr_df = corr_df.iloc[117, :117]
        corr_array = corr_df.abs().values
        print(type(corr_array))
        idx = np.argsort(corr_array)[::-1]
        #前面37个属性相关性太低了，直接去掉。
        #后面4天中各取5个最高的属性，由于前4天的预测值相关性最高，而第5天没有，因此第5天只取4个属性
        feat_idx = idx.tolist()[0:24]
        print("feat_idx", feat_idx)

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


# Training Loop

In [9]:
def trainer(train_loader, valid_loader, model, config, device):
    criterion = nn.MSELoss(reduction='mean')  # Define your loss function, do not modify this.

    # Define your optimization algorithm.
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9, weight_decay=config['L2_regularization'])
    # 为什么使用Adam loss收敛在110+了呢
    # ----学习率太小的了，改成10e-4
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['L2_regularization'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
                                                                     T_0=2, T_mult=2, eta_min=config['learning_rate'])

    # 2、stackoverflow的评论，哪一个是对的
    # In SGD optimizer, L2 regularization can be obtained by weight_decay. But weight_decay and L2 regularization is different for Adam optimizer. More can be read here: openreview.net/pdf?id=rk6qdGgCZ –
    # @Ashish your comment is correct that weight_decay and L2 regularization is different but in the case of PyTorch's implementation of Adam, they actually implement L2 regularization instead of true weight decay. Note that the weight decay term is applied to the gradient before the optimizer step here –

    # writer = SummaryWriter()  # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models')  # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train()  # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()  # Set gradient to zero.
            x, y = x.to(device), y.to(device)  # Move your data to device.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()  # Compute gradient(backpropagation).
            optimizer.step()  # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch + 1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
        scheduler.step()
        mean_train_loss = sum(loss_record) / len(loss_record)
        # writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()  # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record) / len(loss_record)
        print(f'Epoch [{epoch + 1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path'])  # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return


# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    # 'seed': 5201314,  # Your seed number, you can pick your lucky number. :)
    'seed': 5201315,  # Your seed number, you can pick your lucky number. :)
    'select_all': False,  # Whether to use all features.
    'valid_ratio': 0.2,  # validation_size = train_size * valid_ratio
    'n_epochs': 3000,  # Number of epochs.
    'batch_size': 256,
    'learning_rate': 1e-4,
    'early_stop': 400,  # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt',  # Your model will be saved here.
    'L2_regularization': 1e-4
}


# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [11]:
# Set seed for reproducibility
same_seed(config['seed'])

# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days)
# test_data size: 1078 x 117 (without last day's positive rate)
train_data_df = pd.read_csv('./covid.train.csv')
test_data_df  = pd.read_csv('./covid.test.csv')
train_data, test_data = train_data_df.values, test_data_df.values

# print("=====shape():", train_data.shape)
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

#模仿batch_norm做归一化处理
x_min, x_max = x_train.min(axis=0), x_train.max(axis=0)
x_mean = x_train.mean(axis=0)
x_var = ((x_train - x_mean) ** 2).mean(axis=0)
x_train = (x_train - x_mean)/x_var
x_valid = (x_valid - x_mean)/x_var
x_test  = (x_test  - x_mean)/x_var
# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                             COVID19Dataset(x_valid, y_valid), \
                                             COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)


train_data size: (2160, 118) 
valid_data size: (539, 118) 
test_data size: (1078, 117)
<class 'numpy.ndarray'>
feat_idx [101, 85, 69, 53, 104, 88, 105, 72, 89, 56, 73, 40, 57, 41, 103, 102, 87, 86, 71, 70, 55, 54, 39, 38]
number of features: 24


# Start training!

In [ ]:
model = My_Model(input_dim=x_train.shape[1]).to(device)  # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/3000]: 100%|██████████| 9/9 [00:02<00:00,  3.22it/s, loss=102]


Epoch [1/3000]: Train loss: 126.4936, Valid loss: 156.8587
Saving model with loss 156.859...


Epoch [2/3000]: 100%|██████████| 9/9 [00:00<00:00, 96.29it/s, loss=115]


Epoch [2/3000]: Train loss: 122.9007, Valid loss: 139.5769
Saving model with loss 139.577...


Epoch [3/3000]: 100%|██████████| 9/9 [00:00<00:00, 83.08it/s, loss=91.9]


...

Epoch [469/3000]: 100%|██████████| 9/9 [00:00<00:00, 82.75it/s, loss=5.57]


Epoch [469/3000]: Train loss: 4.3305, Valid loss: 1.5387
Saving model with loss 1.539...


Epoch [470/3000]: 100%|██████████| 9/9 [00:00<00:00, 79.77it/s, loss=4.97]


Epoch [470/3000]: Train loss: 4.0371, Valid loss: 1.7425


Epoch [471/3000]:   0%|          | 0/9 [00:00<?, ?it/s, loss=3.43]

# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information. 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)